In [2]:
# Imports & Basic Config
import os
import sqlite3
import shutil
import requests
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from huggingface_hub import hf_hub_url

%matplotlib inline

# Path to your SQLite DB.
KICK_DB_PATH = "../data/kick_data.db"  # Change if needed.

# Hugging Face dataset repository info.
VIDEO_REPO_ID = "PK-Prediction/pk_data"
FRAMES_FOLDER = "frames"
ANNOTATED_FRAMES_FOLDER = "annotated-frames"

# Temporary folder for downloaded images.
LOCAL_TEMP_DIR = "./notebook_temp_viewer"
if os.path.exists(LOCAL_TEMP_DIR):
    shutil.rmtree(LOCAL_TEMP_DIR)
os.makedirs(LOCAL_TEMP_DIR, exist_ok=True)

In [3]:
# Database Queries

def get_all_kick_ids():
    """
    Fetches all kick_id values from the 'kicks' table and returns them in ascending order.
    """
    conn = sqlite3.connect(KICK_DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT kick_id FROM kicks ORDER BY kick_id")
    rows = cur.fetchall()
    conn.close()
    return [r[0] for r in rows]

def get_frames_for_kick(kick_id, annotated=False):
    """
    Returns a list of (frame_id, frame_no, frame_path) for the given kick_id from 'frames' table.
    If annotated=True, only keep frames that actually have an annotated file on Hugging Face.
    If annotated=False, return all frames for that kick.
    """
    conn = sqlite3.connect(KICK_DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        SELECT frame_id, frame_no, frame_path
        FROM frames
        WHERE kick_id=?
        ORDER BY frame_no
    """, (kick_id,))
    raw_rows = cur.fetchall()
    conn.close()

    if not annotated:
        return raw_rows

    # If annotated, filter out frames without an annotated version on Hugging Face.
    filtered = []
    for (fid, fno, fpath) in raw_rows:
        base_name = os.path.basename(fpath)
        ann_name = base_name.replace(".png", "_annotated.png")
        url = hf_hub_url(
            repo_id=VIDEO_REPO_ID,
            filename=f"{ANNOTATED_FRAMES_FOLDER}/{ann_name}",
            repo_type="dataset",
            revision="main"
        )
        resp = requests.get(url)
        if resp.status_code == 200:
            filtered.append((fid, fno, fpath))
    return filtered

def download_image(frame_path, annotated=False):
    """
    Downloads one raw or annotated frame from Hugging Face into LOCAL_TEMP_DIR.
    Returns local file path, or None if download fails.
    """
    base_name = os.path.basename(frame_path)
    if annotated:
        ann_name = base_name.replace(".png", "_annotated.png")
        url = hf_hub_url(
            repo_id=VIDEO_REPO_ID,
            filename=f"{ANNOTATED_FRAMES_FOLDER}/{ann_name}",
            repo_type="dataset",
            revision="main"
        )
        local_name = ann_name
    else:
        url = hf_hub_url(
            repo_id=VIDEO_REPO_ID,
            filename=f"{FRAMES_FOLDER}/{base_name}",
            repo_type="dataset",
            revision="main"
        )
        local_name = base_name

    local_path = os.path.join(LOCAL_TEMP_DIR, local_name)
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(local_path, "wb") as f:
            shutil.copyfileobj(r.raw, f)
        return local_path
    return None

In [4]:
# Viewer Class with ipywidgets.Output()

class KickFrameViewer:
    """
    A class to hold frames for one kick, allowing Next/Prev navigation 
    without losing the interactive widget.
    """
    def __init__(self, frames_list, annotated=False):
        self.index = 0
        self.local_paths = []
        
        # Download each frame (raw or annotated) from HF
        for (fid, fno, fpath) in frames_list:
            path = download_image(fpath, annotated=annotated)
            if path:
                self.local_paths.append((fno, path))

        # Sort by frame_no
        self.local_paths.sort(key=lambda x: x[0])
        self.max_index = len(self.local_paths) - 1

        # ipywidgets.Output for displaying images
        self.out = widgets.Output()

        # Next/Prev buttons
        self.prev_button = widgets.Button(description="Prev", button_style='info')
        self.next_button = widgets.Button(description="Next", button_style='success')
        
        self.prev_button.on_click(self.prev_frame)
        self.next_button.on_click(self.next_frame)

        # Container to display in the notebook
        # We'll show the two buttons + self.out side by side or vertically.
        self.ui = widgets.VBox([
            widgets.HBox([self.prev_button, self.next_button]),
            self.out
        ])
        
        # Show the first frame if any
        self.show_current()
    
    def show_current(self):
        """
        Clears just the output area (self.out) and displays the current image.
        """
        with self.out:
            self.out.clear_output(wait=True)
            if not self.local_paths:
                print("No frames to display.")
                return
            _, local_path = self.local_paths[self.index]
            self._display_image(local_path)
    
    def next_frame(self, _button):
        if self.index < self.max_index:
            self.index += 1
        self.show_current()

    def prev_frame(self, _button):
        if self.index > 0:
            self.index -= 1
        self.show_current()

    def _display_image(self, image_path):
        """
        Uses matplotlib to display one image inline in 'self.out' 
        without clearing the entire notebook cell output.
        """
        plt.close('all')
        if not os.path.exists(image_path):
            print(f"File not found: {image_path}")
            return
        img = plt.imread(image_path)
        plt.figure(figsize=(5,5))
        plt.imshow(img)
        plt.axis('off')
        plt.title(os.path.basename(image_path))
        plt.show()


def create_viewer(kick_id, annotated=False):
    """
    Creates and returns a KickFrameViewer object for the given kick_id and annotation mode.
    """
    frames_list = get_frames_for_kick(kick_id, annotated=annotated)
    if not frames_list:
        return None

    viewer = KickFrameViewer(frames_list, annotated=annotated)
    if not viewer.local_paths:
        return None

    return viewer

In [5]:
# Interactive UI

kick_ids = get_all_kick_ids()
if not kick_ids:
    print("No kicks found in the database.")
else:
    # Dropdowns for the user
    kick_dropdown = widgets.Dropdown(
        options=kick_ids,
        value=kick_ids[0],
        description='Kick ID'
    )
    view_type_dropdown = widgets.Dropdown(
        options=[('Raw Frames', False), ('Annotated Frames', True)],
        value=False,
        description='View Type'
    )

    # We'll store the current viewer in a global variable
    viewer_container = widgets.VBox()  # An empty container to hold the viewer UI

    def update_viewer(*args):
        # Clear the viewer container
        viewer_container.children = ()

        # Build a new viewer for the chosen settings
        k = kick_dropdown.value
        is_annotated = view_type_dropdown.value
        new_viewer = create_viewer(k, annotated=is_annotated)
        if new_viewer is None:
            clear_output(wait=True)
            display(kick_dropdown, view_type_dropdown, viewer_container)
            print(f"No frames to display for kick_id={k}, annotated={is_annotated}.")
        else:
            # If we have a valid viewer, show it in the same cell
            clear_output(wait=True)
            display(kick_dropdown, view_type_dropdown, viewer_container)
            viewer_container.children = (new_viewer.ui,)

    # Observe changes
    kick_dropdown.observe(update_viewer, names='value')
    view_type_dropdown.observe(update_viewer, names='value')

    # Initial display
    display(kick_dropdown, view_type_dropdown, viewer_container)
    update_viewer()


Dropdown(description='Kick ID', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

Dropdown(description='View Type', options=(('Raw Frames', False), ('Annotated Frames', True)), value=False)

VBox()